In [ ]:
# 📦 Instalação de dependências
!pip install selenium beautifulsoup4 pandas --quiet

In [ ]:
# 🔧 Configuração do ambiente
import os
import time
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

categorias = {
    "vinhosDeMesa": "subopt_01",
    "espumantes": "subopt_02",
    "uvasFrescas": "subopt_03",
    "uvasPassas": "subopt_04",
    "sucoDeUva": "subopt_05"
}

anoAtual = 2024
anoInicial = 1970

In [ ]:
# 🚗 Função para iniciar o navegador headless
def iniciarDriver():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--allow-running-insecure-content")
    options.add_argument("--ignore-certificate-errors")
    options.add_argument("--unsafely-treat-insecure-origin-as-secure=http://vitibrasil.cnpuv.embrapa.br")
    return webdriver.Chrome(options=options)

In [ ]:
# 🧠 Escolher intervalo de anos
def definirIntervaloAnos():
    print("\n📌 Escolha o intervalo de anos:")
    print("1 - Último ano (2024)")
    print("2 - Um ano específico")
    print("3 - Últimos 5 anos")
    print("4 - Últimos 10 anos")
    print("5 - Todos os anos (1970 a 2024)")
    escolha = input("Digite o número da opção desejada: ")
    if escolha == "1": return [anoAtual]
    elif escolha == "2":
        ano = int(input("Digite o ano desejado (ex: 2005): "))
        if ano < anoInicial or ano > anoAtual:
            raise ValueError("Ano fora do intervalo permitido.")
        return [ano]
    elif escolha == "3": return list(range(anoAtual - 4, anoAtual + 1))
    elif escolha == "4": return list(range(anoAtual - 9, anoAtual + 1))
    elif escolha == "5": return list(range(anoInicial, anoAtual + 1))
    else: raise ValueError("Opção inválida.")

In [ ]:
# 🌎 Extrair dados por categoria e anos
def extrairPorCategoriaEAnos(driver, nomeCategoria, subopcao, anos):
    dadosColetados = []
    for ano in anos:
        url = f"http://vitibrasil.cnpuv.embrapa.br/index.php?ano={ano}&opcao=opt_05&subopcao={subopcao}"
        print(f"🔄 Coletando: {nomeCategoria} - {ano}")
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tabela = soup.find("table", {"class": "tb_base tb_dados"})
        if not tabela:
            print(f"⚠️ Tabela não encontrada para {nomeCategoria} em {ano}")
            continue
        linhas = tabela.find("tbody").find_all("tr")
        for linha in linhas:
            cels = linha.find_all("td")
            if len(cels) >= 3:
                pais = cels[0].text.strip()
                qtd = cels[1].text.strip().replace('.', '').replace(',', '.')
                valor = cels[2].text.strip().replace('.', '').replace(',', '.')
                try:
                    dadosColetados.append([
                        ano, pais, nomeCategoria,
                        float(qtd) if qtd != '-' else 0.0,
                        float(valor) if valor != '-' else 0.0
                    ])
                except:
                    continue
    return dadosColetados

In [ ]:
# 💾 Salvar dados em CSV
def salvarCsv(dados, nomeCategoria):
    os.makedirs("dadosImportacao", exist_ok=True)
    df = pd.DataFrame(dados, columns=["Ano", "Pais", "Categoria", "QuantidadeKg", "ValorUsd"])
    df.to_csv(f"dadosImportacao/{nomeCategoria}.csv", index=False)
    print(f"✅ CSV salvo: dadosImportacao/{nomeCategoria}.csv")
    return df

In [ ]:
# 🚀 Rodar coleta completa
anosSelecionados = definirIntervaloAnos()
driver = iniciarDriver()
todosDados = []
for categoriaNome, codigo in categorias.items():
    dados = extrairPorCategoriaEAnos(driver, categoriaNome, codigo, anosSelecionados)
    if dados:
        salvarCsv(dados, categoriaNome)
        todosDados.extend(dados)
    else:
        print(f"⚠️ Nenhum dado encontrado para: {categoriaNome}")
driver.quit()
print("🏁 Finalizado!")

In [ ]:
# 👀 Visualizar todos os dados
df_total = pd.DataFrame(todosDados, columns=["Ano", "Pais", "Categoria", "QuantidadeKg", "ValorUsd"])
df_total.head()